In [99]:


import geopandas as gpd
import numpy as np

from feature.utils import timing
from parsers.utils import pcloud
from parsers.utils import raster

In [100]:
# class BufferableGeoDataFrame(gpd.GeoDataFrame):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__()
# 
#     def buffer(self, *args, **kwargs) -> None:
#         ...

@timing
def __init__(filename: str) -> gpd.GeoDataFrame:
    return gpd.read_file(filename)


@timing
# TODO: Review the various buffer options.
# TODO: Rewrite this function as an appropriate class method.
def buffer(objs: gpd.GeoDataFrame, dist: float = 10) -> None:
    objs["geometry"] = objs["geometry"].buffer(dist)

## Step 1
Load the point cloud into memory.

In [101]:
pc = pcloud.PointCloud("../aula.LAZ")

func __init__ :-> 352 ms


## Step 2
Load the building footprints into memory.

In [102]:
fps = __init__("C:/Documents/RoofSense/data/temp/9-284-556.buildings.gpkg")

func __init__ :-> 230 ms


## Step 3
Buffer the footprints by 10 m.

NOTE: The buffer distance is selected such that most if not all the natural neighbors of 
      the points along the footprint edges are included in the subsequent steps.

In [103]:
buffer(fps)

func buffer :-> 90 ms


## Step 4
Intersect the point cloud with the buffers.

In [104]:
ids = pc.intersect(fps)

func intersect :-> 33833 ms


In [105]:
pc.las.points = pc.las.points[ids]

## Step 5
Compute the Delaunay triangulation of the remaining points.

In [106]:
pc.triangulate()

func triangulate :-> 9182 ms


## Step 6
 Estimate the slope field of the corresponding network.

TODO

In [107]:
# TOSELF: Promote this argument to an environment variable?
CELL_SIZE = 0.25

grid = raster.Raster(CELL_SIZE, pc.dt.get_bbox())

# FIXME: Integrate this block into the Raster initializer.
# Construct the grid.
# TODO: Add documentation.
rows, cols = np.mgrid[grid.height - 1:-1:-1, 0:grid.width]
# TODO: Add documentation.
xx = grid._bbox[0] + CELL_SIZE * (cols + 0.5)
yy = grid._bbox[1] + CELL_SIZE * (rows + 0.5)
# TODO: Add documentation.
cells = np.column_stack([xx.ravel(), yy.ravel()])

TODO

In [ ]:
# TODO: Speed up this block.
# TOSELF: Compute the slope for all faces to avoid additional calls to dt.locate?
valid_cells = []
valid_faces = []
for i, center in enumerate(cells):
    try:
        valid_faces.append(pc.dt.locate(*center))
    except Exception:
        continue
    valid_cells.append(i)

# TOSELF: Discard duplicate faces?
# valid_faces = np.unique(valid_faces, axis=0)

In [ ]:
# TODO: Add documentation.
dt_pts = pc.dt.points
valid_faces = np.array(valid_faces)
v1 = dt_pts[valid_faces[:, 0]]
v2 = dt_pts[valid_faces[:, 1]]
v3 = dt_pts[valid_faces[:, 2]]

u = v2 - v1
v = v3 - v1

n = np.cross(u, v)
n = n / np.linalg.norm(n, axis=1)[:, None]

z = np.array([0, 0, 1])
s = np.degrees(np.arccos(np.dot(n, z)))

In [ ]:
grid._data[np.divmod(valid_cells, grid.width)] = s
grid.save("../aula_slpe.tiff")

## Step 6
 Rasterize the slope and reflectance fields of the corresponding network.

In [ ]:
# Interpolate the field values at the cell centers.
vals = pc.dt.interpolate({"method": "Laplace"}, cells)

In [ ]:
# Populate the raster.
grid._data = vals.reshape((grid.height, grid.width))

# Save the raster.
grid.save("../aula_elev.tiff")

In [112]:
import math

import numpy as np
import startinpy


def laplace(dt: startinpy.DT, x: float, y: float) -> tuple[list[int], list[float]]:
    num = dt.number_of_vertices()
    if not num:
        raise RuntimeError("The triangulation does not contain any vertices.")

    try:
        dt.locate(x, y)
    except Exception:
        raise ValueError(
            f"The interpolation point ({x, y}) is located outside the convex hull of "
            f"the triangulation."
        )

    idx = dt.insert_one_pt(x, 
                           y, 
                           0)
    # The point was inserted at the same location as an existing vertex.
    if dt.number_of_vertices() == num:
        # return [idx], [1]
        return dt.points[idx][2]
    if dt.is_vertex_convex_hull(idx):
        dt.remove(idx)
        raise ValueError(
            f"The interpolation point ({x, y}) is located on the boundary of the "
            f"convex hull of the triangulation."
        )

    pts = dt.points
    trs = dt.incident_triangles_to_vertex(idx)


    centers = []
    contrib = []
    for tr in trs:
        centers.append(circumcentre(pts[tr[0]], 
                                    pts[tr[1]], 
                                    pts[tr[2]]))
        # print(tr)
        contrib.append(tr[1])
    # print(centers)
    centers.append(centers[0])
    print(trs)
    print(len(contrib))
    print(len(centers))
    
    # contrib.append(contrib[0])
    
    # print(len(centers),len(contrib))

    weights = []
    for i in range(len(centers) - 1):
        dx = centers[i][0] - centers[i + 1][0]
        dy = centers[i][1] - centers[i + 1][1]
        e = math.hypot(dx, dy)

        dx = pts[idx][0] - pts[contrib[i]][0]
        dy = pts[idx][1] - pts[contrib[i]][1]
        w = math.hypot(dx, dy)

        weights.append(e / w)
    print(i)
    print()

    dt.remove(idx)
    # return contrib,weights
    con = []
    for c in contrib:
        con.append(pts[c][2])
    weights = np.array(weights)
    con = np.array(con)
    return np.sum(con * weights) / np.sum(weights)

    
    
    
def circumcentre(a, b, c):
    A = det3x3(a[0], a[1], 1, b[0], b[1], 1, c[0], c[1], 1)
    B = det3x3(
        a[0] * a[0] + a[1] * a[1],
        a[1],
        1,
        b[0] * b[0] + b[1] * b[1],
        b[1],
        1,
        c[0] * c[0] + c[1] * c[1],
        c[1],
        1,
    )
    C = det3x3(
        a[0] * a[0] + a[1] * a[1],
        a[0],
        1,
        b[0] * b[0] + b[1] * b[1],
        b[0],
        1,
        c[0] * c[0] + c[1] * c[1],
        c[0],
        1,
    )
    x = +B / (2 * A)
    y = -C / (2 * A)
    return x, y


def det3x3(ax, ay, az, bx, by, bz, cx, cy, cz):
    temp1 = ax * (by * cz - bz * cy)
    temp2 = ay * (bx * cz - bz * cx)
    temp3 = az * (bx * cy - by * cx)
    return temp1 - temp2 + temp3


In [113]:


# own_vals = []
for i,cell in enumerate(cells):
    # if i==9000:
        # break
    try:
        expect=pc.dt.interpolate({"method": "Laplace"},[cell])
        actual=laplace(pc.dt, *cell)
        print(expect,actual)
    except (RuntimeError, ValueError) as e:
        pass
# grid._data = vals.reshape((grid.height, grid.width))
# grid.save("../aula_elev_own.tiff")

[0.] 0.0
[[1922812 1922811 1886338]
 [1922812 1886338 1886281]
 [1922812 1886281 1895099]
 [1922812 1895099 1895051]
 [1922812 1895051 1895006]
 [1922812 1895006 1922811]]
6
7
5

[-0.26279117] -0.5187361141779409
[[1922812 1922811 1895099]
 [1922812 1895099 1895051]
 [1922812 1895051 1895006]
 [1922812 1895006 1922811]]
4
5
3

[-0.48812469] -0.48680404200942307
[[1922812 1895051 1886229]
 [1922812 1886229 1922469]
 [1922812 1922469 1886176]
 [1922812 1886176 1895006]
 [1922812 1895006 1895051]]
5
6
4

[-0.58373458] -0.6006467430950808
[[1922812 1886176 1922469]
 [1922812 1922469 1886124]
 [1922812 1886124 1894960]
 [1922812 1894960 1922514]
 [1922812 1922514 1886176]]
5
6
4

[-0.59512042] -0.5932530132033759
[[1922812 1894960 1911686]
 [1922812 1911686 1886072]
 [1922812 1886072 1911788]
 [1922812 1911788 1894918]
 [1922812 1894918 1894960]]
5
6
4

[-0.61290103] -0.6183597335277072
[[1922812 1911788 1894917]
 [1922812 1894917 1922599]
 [1922812 1922599 1911886]
 [1922812 1911886 189487

KeyboardInterrupt: 